In [ ]:
import re
import os
import string
import operator
from collections import OrderedDict, defaultdict
from fnmatch import fnmatch

sortsize = 100000000
min_size = 17
char = [i for i in range(10)] + [j for j in string.ascii_lowercase]
bar8 = get_default_barcode() # eg, bar['TGCGTGAT']=168

def main():
    sample = input('Fastq sample name:\n') # eg, RL032816
    barcode = input("Barcodes separated by comma (or 'barcode_file'):")
    
    if barcode != 'barcode_file':
        barcodes = re.split('\D+', barcode.strip())
        samples = input_sam(barcodes) # eg, samples['168'] = wt
    else:
        barcodes, samples = input_file()
    
    zp = input('Is the file zipped (compressed)? - yes or no\n')
    zp = zp.strip()
    
    #unzip if necessary
    if re.search(r'yes', zp, re.IGNORECASE):
        unzip_fastq(sample)
        
    # get the largest number of the files
    fnum =get_file_no(sample)
    
    # remove 3' linker
    for i in range(1,fum+1):
        trim53(sample, i)
    bar8.clear()
    
    #os.system('rm -r fastq/'+sample)
    
    # make uni file and zip fastq
    for bc in samples.keys():
        sam = samples[bc]
        
        if os.path.getsize(sam) > sortsize: # returns size in bytes
            sort_uni(sam)
        else:
            uni(sam)
    
    # zip individual fastq after barcoding
    zip_file(samples)
        
        
def trim53(sam, fn):
    s = samples
    
    # get read size
    with open('fastaq/'+sam+'/'+sam+'_pair1_'+str(fn)+'.fastq') as f:
        line_num = 0
        size = 0
        for line in f:
            line = line.rstrip()
            line_num += 1
            if (line_num == 2):
                size = len(line)
                print ('read size '+ str(size) + '\n')
                break
    
    # get the index
    with open('fastaq/'+sam+'/'+sam+'_pair2_'+str(fn)+'.fastq') as f:
        idx = collections.OrderedDict()
        line_num = 0
        for line in f:
            line_num += 1
            if line_num % 4 == 2:
                line = line.rstrip()
                idx[line_num] = line
                
                
    # remove 3' linker
    buf = []
    count = 61000000
    with open('fastaq/'+sam+'/'+sam+'_pair1_'+str(fn)+'.fastq') as f:
        line_num = 0
        for line in f:
            line_num += 1
            mod = line_num % 4
            line = line.rstrip()
            if mod == 2:
                buf.append(idx[line_num])
                buf.append(line)
            elif mod == 0:
                buf.append(line)
            
            if (line_num % 4000000 == 0):
                ano = len(buf) - 1
                rse = OrderedDict(defaultdict(list))
                rfq = OrderedDict(defaultdict(list))
                for i in range(0,ano,3):
                    bc = buf[i] # barcode
                    se = buf[i+1] # read sequence
                    qua = buf[i+2] # read quality
                    #bar = '?*&%#@!~'
                    if bc in bar8.keys():
                        bar = bar8[bc] # 168, 169

                        rfq[bar].append(se+'\n+\n'+qua+'\n')
                        
                        # trim 3' end
                        if size == 36:
                            if (re.search(r'^(.*)AGATCG', se)):
                                mch = re.search(r'^(.*)AGATCG', se)
                                se = mch.group[1]
                            else:
                                se = re.sub(r'AGATC|AGAT|AGA|AG$', r'', se)
                                
                        elif size == 50:
                            if (re.search(r'^(.*)AGATCGGAA', se)):
                                mch = re.search(r'^(.*)AGATCGGAA', se)
                                se = mch.group[1]
                            else:
                                se = re.sub(r'AGATCGGA|AGATCGG|AGATCG|AGATC|AGAT|AGA|AG$', r'', se)
                        if len(se) >= min_size and re.search(r'\.|N', se) == False:
                            rse[bar].append(se)
                for bar in rse.keys():
                    if bar not in s.keys(): # what is s?
                        continue
                    sss = s[bar]
                    fh = open ('fastq/'+sss+'/'+sss+'_'+min_size+',txt', 'at')
                    for se in rse[bar]:
                        fh.write(se+'\n')
                    fh.close()
        
                for bar in rfq.keys():
                    if bar not in s.keys(): # what is s?
                        continue
                    sss = s[bar]
                    fh = open ('fastq/'+sss+'/'+sss+'_raw.fastq', 'at')
                    for fq in rfq[bar]:
                        count += 1
                        seqid = convert_to_36(count)
                        fh.write('@'+id+'\n'+fq)
                    fh.close()
        
                buf.clear()
                rfq.clear()
                rse.clear()

        
    ### the last < 4000000 reads
    ano = len(buf) - 1
    rse = OrderedDict(defaultdict(list))
    rfq = OrderedDict(defaultdict(list))
    for i in range(0,ano,3):
        bc = buf[i] # barcode
        se = buf[i+1] # read sequence
        qua = buf[i+2] # read quality
        bar = '?*&%#@!~'
        if bc in b.keys():
            bar = b[bc] # 168, 169??
                        
            rfq[bar].append(se+'\n+\n'+qua+'\n')
                        
            # trim 3' end
            #yes = 1
            if size == 36:
                if (re.search(r'^(.*)AGATCG', se)):
                    mch = re.search(r'^(.*)AGATCG', se)
                    se = mch.group[1]
                else:
                    se = re.sub(r'AGATC|AGAT|AGA|AG$', r'', se)
                                
            elif size == 50:
                if (re.search(r'^(.*)AGATCGGAA', se)):
                    mch = re.search(r'^(.*)AGATCGGAA', se)
                    se = mch.group[1]
                else:
                    se = re.sub(r'AGATCGGA|AGATCGG|AGATCG|AGATC|AGAT|AGA|AG$', r'', se)
            if len(se) >= min_size and re.search(r'\.|N', se) == False:
                rse[bar].append(se)
    for bar in rse.keys():
        if bar not in s.keys(): # what is s?
            continue
        sss = s[bar]
        fh = open ('fastq/'+sss+'/'+sss+'_'+min_size+',txt', 'at')
        for se in rse[bar]:
            fh.write(se+'\n')
        fh.close()
        
    for bar in rfq.keys():
        if bar not in s.keys(): # what is s?
            continue
        sss = s[bar]
        fh = open ('fastq/'+sss+'/'+sss+'_raw.fastq', 'at')
        for fq in rfq[bar]:
            count += 1
            seqid = convert_to_36(count)
            fh.write('@'+id+'\n'+fq)
        fh.close()
        
    buf.clear()
    rfq.clear()
    rse.clear()
    s.clear()

def uni(sam):
    #sample = sam.pop(0)
    i = 61000000
    u = OrderedDict()
    with open('fastq/'+sam+'/'+sam+'_'+min_size+'.txt') as f:
        for line in f:
            u[line] += 1
    os.remove('fastq/'+sam+'/'+sam+'_'+min_size+'.txt')
    
    fh = open('fastq/'+sam+'/'+sam+'_'+min_size+'_uni.txt', 'wt')
    
    for k, v in sorted(u.items(), key = operator.itemgetter(1)):
        i += 1
        seqid = convert_to_36(i)
        fh.write('>'+seqid+'_x'+v+'\n'+k)
        
    fh.close()
    
    u.clear()
    
    
def sort_uni(sam):
    se = '@#$%^&*'
    re = 0
    bs = 61000000
    os.system('sort -k1,1 fastaq/'+sam+'/'+sam+'_'+min_size+'.txt > fastaq/'+sam+'/'+sam+'_'+min_size+'.sorted')
    
    fh = open ('fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.tmp')
    with open ('fastaq/'+sam+'/'+sam+'_'+min_size+'.sorted') as f:
        for line in f:
            if (line == se):
                re += 1
            else:
                if (se != '@#$%^&*'):
                    fh.write(re+'\t'+se)
                se = line
                re = 1
        if (se != '@#$%^&*'):
            fh.write(re+'\t'+se)
    fh.close()
    
    os.remove('fastaq/'+sam+'/'+sam+'_'+min_size+'.sorted')
    os.remove('fastaq/'+sam+'/'+sam+'_'+min_size+'.txt')
    
    os.system('sort -k1,1 -r -n fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.tmp > fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.sorted')
    
    fh = open ('fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.txt')
    with open ('fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.sorted') as f:
        for line in f:
            bs += 1
            re, se = split()
            seqid = convert_to_36(bs)
            fh.write('>'+seqid+'_x'+re+'\n'+se)
    
    os.remove('fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.tmp')
    os.remove('fastaq/'+sam+'/'+sam+'_'+min_size+'_uni.sorted')
    
    
    
### default barcodes
def get_default_barcode():
    b['TGCGTGAT']=168; b['AGACATCG']=169; b['ACGCCTAA']=170;
    b['GGTGGTCA']=171; b['CGCACTGT']=172; b['AAATTGGC']=173;
    b['CCGATCTG']=174; b['CATCAAGT']=175; b['AACTGATC']=176;
    b['AAAAGCTA']=177; b['CCGTAGCC']=178; b['ATTACAAG']=179;
    b['ACTTGACT']=180; b['GTGGAACT']=181; b['TGGGACGG']=182;
    b['TGCTCTAC']=183; b['GAGCGGAC']=184; b['AGTTTCAC']=185;
    b['CCGGCCAC']=186; b['GGCGAAAC']=187; b['TACGTACG']=188;  
    b['CGCCACTC']=189; b['TAGCTACC']=190; b['TCGCTCAT']=191;
    b['GCCTTTTG']=192; b['GGTAGTTG']=193; b['ATCCGGTG']=194;
    b['TATGAGTG']=195; b['CGAAAATG']=196; b['GCTGTTGG']=197;
    b['GAATTCCG']=290; b['GAGTATAG']=291; b['CCTCTGAG']=292;
    b['TCGTCGTC']=293; b['TACGATTA']=294; b['TAGCTGTA']=295;
    b['CTATTATA']=383; b['CAGAATGA']=384; b['GTTCGGGA']=385;
    b['GGCTTCGA']=386; b['CATGCCGA']=387; b['GGCCGAAG']=388;
    b['TGTAACAA']=389; b['ATTGGATT']=390; b['TTAAAGGA']=391;
    b['GATTGTAC']=392; b['GAGTCAGG']=393; b['GTGTGCTA']=394;
    
    return b
    
    
### get the barcodes and sample names, copy the script into folders
# make directories, save [(barcode,sample)] into smp, and/or barcodes into bc
def input_sam(barcodes):
    #script = __file__
    smp = collections.OrderedDict()
    print ('\nTo skip a sample, type Enter.\n')
    for barcode in barcodes:
        sa = input('Barcode', barcode, 'sample name:')
        sa = sa.rstrip()
        if not re.search(r'\w', sa):
            print ('Barcode', barcode, 'skipped\n')
            next
        smp[barcode] = sa
        
        if os.path.isdir('fastq/'+sa):
            print (sa+'exists.\n')
            rmd = input('Delete, yes or no:')
            rmd = rmd.rstrip()
            if re.search(r'yes', rmd, re.IGNORECASE):
                os.system('rm -r fastq/'+sa)
            else:
                exit()
        os.makedirs('fastq/'+sa)
        #os.system('cp '+ script + ' fastq/'+sa)
    
    return smp
  
# barcode file    
def input_file():
    #script = __file__
    bc = []
    smp = collections.OrderedDict()
    fl = input('Barcode file name:')
    fl = fl.rstrip()
    with open (fl) as f:
        for line in f:
            line = line.rstrip()
            b, n = line.split()
            bc.append(b)
            smp[b] = n
            
        if os.path.isdir('fastq/'+n):
            print (sa+'exists.\n')
            rmd = input('Delete, yes or no:')
            rmd = rmd.rstrip()
            if re.search(r'yes', rmd, re.IGNORECASE):
                os.system('rm -r fastq/'+n)
            else:
                exit()
        os.makedirs('fastq/'+n)
        #os.system('cp '+ script + ' fastq/'+n)
            
        return bc, smp
            
### get the largest spliting number    
def get_file_no(sam):
    fno = 0
    path = 'fastq/'+sam+'/'
    fqfiles = [name for name in os.listdir(path) if fnmatch(name, '*.fastq')]
    for f in fqfiles:
        ff = re.search(r'_(\d+).fastq$', f)
        mch = int(ff.group[1])
        if mch > fno:
            fno = mch
    return fno
    
    
### unzip raw sequencing data    
def unzip_fastq(sam):
    path = 'fastq/'+sam+'/'
    fqfiles = [name for name in os.listdir(path) if name.endswith('.gz')]
    for f in fqfiles:
        os.system('gzip -d '+path+f)
    
    
### zip raw data after barcoding
import os
def zip_fastq(smp):
    for bc in smp.keys():
        sample = smp[bc]
        infile = 'fastq/' + sample + '/' + sample + '_fastq'
        outfile = 'fastq/' + sample + '/' + sample + '_fastq.tar.gz'
        os.system('tar -cvzf ' + outfile + ' ' + infile)
        os.system('rm ' + infile)
      
    
### convert to 36 based number
# 6-characters, 36**5 = 60466176 < 61000000    
def convert_to_36(n):
    out = char[n % 36] # 0 to 35
    n = int(n/36)
    
    while True:
        out = str(char[n % 36]) + out
        n = int(n/36)
        if n < 1:
            break
    return out
 
    
if __name__ == '__main__':
    main()

In [4]:
def get_default_barcode():
    b = collections.OrderedDict()
    b['TGCGTGAT']=168; b['AGACATCG']=169; b['ACGCCTAA']=170;
    b['GGTGGTCA']=171; b['CGCACTGT']=172; b['AAATTGGC']=173;
    b['CCGATCTG']=174; b['CATCAAGT']=175; b['AACTGATC']=176;
    b['AAAAGCTA']=177; b['CCGTAGCC']=178; b['ATTACAAG']=179;
    b['ACTTGACT']=180; b['GTGGAACT']=181; b['TGGGACGG']=182;
    b['TGCTCTAC']=183; b['GAGCGGAC']=184; b['AGTTTCAC']=185;
    b['CCGGCCAC']=186; b['GGCGAAAC']=187; b['TACGTACG']=188;  
    b['CGCCACTC']=189; b['TAGCTACC']=190; b['TCGCTCAT']=191;
    b['GCCTTTTG']=192; b['GGTAGTTG']=193; b['ATCCGGTG']=194;
    b['TATGAGTG']=195; b['CGAAAATG']=196; b['GCTGTTGG']=197;
    b['GAATTCCG']=290; b['GAGTATAG']=291; b['CCTCTGAG']=292;
    b['TCGTCGTC']=293; b['TACGATTA']=294; b['TAGCTGTA']=295;
    b['CTATTATA']=383; b['CAGAATGA']=384; b['GTTCGGGA']=385;
    b['GGCTTCGA']=386; b['CATGCCGA']=387; b['GGCCGAAG']=388;
    b['TGTAACAA']=389; b['ATTGGATT']=390; b['TTAAAGGA']=391;
    b['GATTGTAC']=392; b['GAGTCAGG']=393; b['GTGTGCTA']=394;
    
    return b

barcodes = get_default_barcode()
for k, v in barcodes.items():
    print (k, v)

TGCGTGAT 168
AGACATCG 169
ACGCCTAA 170
GGTGGTCA 171
CGCACTGT 172
AAATTGGC 173
CCGATCTG 174
CATCAAGT 175
AACTGATC 176
AAAAGCTA 177
CCGTAGCC 178
ATTACAAG 179
ACTTGACT 180
GTGGAACT 181
TGGGACGG 182
TGCTCTAC 183
GAGCGGAC 184
AGTTTCAC 185
CCGGCCAC 186
GGCGAAAC 187
TACGTACG 188
CGCCACTC 189
TAGCTACC 190
TCGCTCAT 191
GCCTTTTG 192
GGTAGTTG 193
ATCCGGTG 194
TATGAGTG 195
CGAAAATG 196
GCTGTTGG 197
GAATTCCG 290
GAGTATAG 291
CCTCTGAG 292
TCGTCGTC 293
TACGATTA 294
TAGCTGTA 295
CTATTATA 383
CAGAATGA 384
GTTCGGGA 385
GGCTTCGA 386
CATGCCGA 387
GGCCGAAG 388
TGTAACAA 389
ATTGGATT 390
TTAAAGGA 391
GATTGTAC 392
GAGTCAGG 393
GTGTGCTA 394


In [54]:
fq = [name for name in os.listdir('.') if name.endswith('.gz')]
print (fq)

['test.txt.tar.gz']


In [59]:
from fnmatch import fnmatch
path = 'base script/'
fqfiles = [name for name in os.listdir(path) if fnmatch(name, '*.pl')]
print (fqfiles)

['step1.0_linker1_split_sample_fastq_from_fastq_fast_mlcore_080813.pl', 'step1.0_linker2_split_sample_fastq_from_fastq_fast_mlcore_080813.pl', 'step1.1_extract_size_with_1st_060311.pl', 'step1.2_linker1_split_sample_fastq_from_qseq_fast_mlcore_080813.pl', 'step1.3_linker1_split_sample_fastq_from_qseq_fast_mlcore_for_bad_quality_080813.pl', 'step2.0_bowtie_match_mlcore_080813.pl', 'step2.1_bowtie_match_mlcore_080813.pl', 'step3.0_hist_WS215_bowtie_1st_nt_060311.pl', 'step3.1_hist_WS215_bowtie_1st_nt_060311.pl', 'Step4_make_mysql_database.pl', 'step6_extend_match.pl']


In [51]:
import os
def zip_fastq(infile):
    outfile = infile+'.tar.gz'
    os.system('tar -cvzf '+outfile+' '+infile)
    os.system('rm '+infile)
    
zip_fastq('test.txt')

In [61]:
x = 's n'
m , z = x.split()
print (m,z)

s n


In [69]:
import re
rmd = 'yE2s   '
for i in range(10):
    print (i)
    if re.search(r'yes', rmd, re.IGNORECASE):
        print ('match')
    else:
        exit()

0
1
2
3
4
5
6
7
8
9


In [1]:
import collections
import re
import os
def input_sam(barcodes):
    smp = collections.OrderedDict()
    print ('\nTo skip a sample, type Enter.\n')
    for barcode in barcodes:
        sa = input('Barcode '+barcode+' sample name:')
        if not re.search(r'\w', sa):
            print ('Barcode', barcode, 'skipped\n')
            next
        smp[barcode] = sa
        
        
        if os.path.isdir('fastq/'+sa):
            print (sa+'exists.\n')
            rmd = input('Delete, yes or no:')
            rmd = rmd.rstrip()
            if re.search(r'yes', rmd, re.IGNORECASE):
                os.system('rm -r fastq/'+sa)
            else:
                exit()
        os.makedirs('fastq/'+sa)
        #os.system('cp $0 fastq/'+sa)
    
    return smp

barcodes = ['172', '173']
samples = input_sam(barcodes)
print (samples['172'])


To skip a sample, type Enter.

Barcode 172 sample name:wt
Barcode 173 sample name:mut
wt


In [22]:
class Song(object):

    def __init__(self, lyrics):
        self.lyrics = lyrics

    def sing_me_a_song(self):
        for line in self.lyrics:
            print line     

[1, 2, 3]


In [32]:
#### convert to 36 based number
# 6-characters, 36**5 = 60466176 < 61000000
import string
char = [i for i in range(10)] + [j for j in string.ascii_lowercase]

def convert_to_36(n):
    out = char[n % 36] # 0 to 35
    n = int(n/36)
    
    while True:
        out = str(char[n % 36]) + out
        n = int(n/36)
        if n < 1:
            break
    return out

convert_to_36(61000000)

'10bfwg'

In [ ]:
out = 'x'
out = 'x'+out
print (out)

In [40]:
for i in range(1,7):
    print (60466176/36**i)

1679616.0
46656.0
1296.0
36.0
1.0
0.027777777777777776


In [37]:
36**5

60466176

In [7]:
100 % 4000000

100

In [15]:
import string
n = [i for i in range(10)] + [j for j in string.ascii_lowercase]
n

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [1]:
barcode = '172, 173   '
barcode.rstrip()

'172, 173'

In [5]:
import re
barcodes = re.split('\D+', barcode.strip())
barcodes

['172', '173']

In [3]:

with open('ERR478129_1.fastq') as f:
    line_num = 0
    for line in f:
        line = line.rstrip()
        line_num += 1
        if (line_num == 2):
            print ('sequence:'+line)
            size = len(line)
            print ('read size '+ str(size))
            break
            
            


sequence:TTTGGGCCGGTGGAAGAGAAGGGAAAGAGGTGTGTTGTGGGTAAGATTTTGCAACATCAATCTGCAGACTGCAACTAGTGAGTATTACGACCCACT
read size 96


In [ ]:
with open('ERR478129_1.fastq') as f:
    idx = collections.OrderedDict()
    line_num = 0
    for line in f:
        line_num += 1
        if line_num % 4 == 2:
            line = line.rstrip()
            idx[line_num] = line
            
for k, v in idx.items():
    print (k, v)

In [21]:
from collections import defaultdict, OrderedDict
rse = OrderedDict(defaultdict(list))
rse['x'] = [1,2,3,4]
rse['y'] = [1,5,3,4]
for x in rse.keys():
    print (rse[x])

[1, 2, 3, 4]
[1, 5, 3, 4]


In [16]:
from collections import defaultdict
rfq = defaultdict(list)
type(rfq)

collections.defaultdict